Mark Belikov 4/21/2024 

How do reviews on app change users intention of downloading the app.

The problem is to find what makes a game or app have good reviews on the app store

Solution is to load dataset and clean it to see what factors contribute to an app getting good ratings. 

In [1]:
!pip install textblob
!pip install wordcloud
import pandas as pd
import nltk
from textblob import TextBlob
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from urllib.parse import quote

In [2]:
!pip install requests
import requests

In [3]:
api_key = "6JiP6oz0qbzaGsYXl32MCnxiG9Jm7mES6nOtf0Lv6eEWNIB500PZ1Z92JhqWmGdsRCR1-bydOfXZw234UAIozBzTZw7pTcuS5uccF26BvRgij5pArgdeBiOMCaQmZnYx"

In [4]:
API_HOST = 'https://api.yelp.com'
SEARCH_PATH = '/v3/businessess/search'
BUSINESS_PATH = '/v3/businesses/'

In [5]:
DEFAULT_TERM = 'software'
DEFAULT_LOCATION = 'Jersey City, NJ'
SEARCH_LIMIT = 20

In [6]:
def request(host, path, api_key, url_params=None):
    url_params = url_params or {}
    url = '{0}{1}'.format(host, quote(path.encode('utf8')))
    headers = {
        'Authorization': 'Bearer %s' % api_key
    }
    
    print(u'Querying {0} ...'.format(url))
    
    response = requests.request('GET', url, headers=headers, params=url_params)
    
    return response.json()

def search(api_key, term, location):
    url_params = {
        'term': term.replace(' ', '+'),
        'location': location.replace(' ', '+'),
        'limit': SEARCH_LIMIT
    }
    return request(API_HOST, SEARCH_PATH, api_key, url_params=url_params)

def get_business(api_key, business_id):
    business_path = BUSINESS_PATH + business_id
    
    return request(API_HOST, business_path, api_key)

def query_api(term, location):
    response = search(API_KEY, term, location)
    
    businesses = response.get('businesses')
    
    if not businesses:
        print(u'No businesses for {0} in {1} found.'.format(term, location))
        return
    
    business_id = businesses[0]['id']
    
    print(u'{0} businesses found, querying business infor ' \
          'for the top result "{1}" ...'.format(
                len(businesses), business_id))
    response = get_business(API_KEY, business_id)
          
    print(u'Result for business "{0}" found:'.format(business_id))
    pprint.pprint(response, indent=2)

In [7]:
response = search(api_key, DEFAULT_TERM, 'Jersey City, NJ')

Querying https://api.yelp.com/v3/businessess/search ...


In [8]:
response

{'error': {'code': 'NOT_FOUND', 'description': 'Resource could not be found.'}}

In [ ]:
for business in response['businesses']:
    print(response)
    print('Business Name: ' + business['name'])
    print('Rating: ' + str(business['rating']))
    print('Review Count: ' + str(business['review_count']))
    print('Reviews: ')
    BUS_REVIEW='/v3/businesses/{}/reviews'.format(business['id'])
    
    reviews = request(API_HOST,BUS_REVIEW, API_KEY)
    for review in reviews['reviews']:
        print(' - ' + review['text'])
        
        blob = TextBlob(review['text'])
        sentiment = blob.sentiment.polarity
        if sentiment > 0:
            print('  Sentiment: Positive')
        elif sentiment < 0:
            print('  Sentiment: Negative')
        else:
            print(' Sentminet: Neutral')